In [ ]:
import numpy as np
import torch
import os
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [140]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [141]:
class BrainTumorDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)

# CNN Model class
class CNNmodel(nn.Module):
    def __init__(self):
        super(CNNmodel, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.lin1 = nn.Linear(128 * 32 * 32, 256)
        self.lin2 = nn.Linear(256, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2)

        x = x.view(x.size(0), -1)
        x = torch.relu(self.lin1(x))
        x = self.lin2(x)
        x = self.sigmoid(x)

        return x


In [142]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Use smaller images for performance
    transforms.ToTensor()
])

dataPaths = ["PMRAM Bangladeshi Brain Cancer - MRI Dataset/PMRAM Bangladeshi Brain Cancer - MRI Dataset/Augmented Data/Augmented/512Glioma",
             "PMRAM Bangladeshi Brain Cancer - MRI Dataset/PMRAM Bangladeshi Brain Cancer - MRI Dataset/Augmented Data/Augmented/512Meningioma",
             "PMRAM Bangladeshi Brain Cancer - MRI Dataset/PMRAM Bangladeshi Brain Cancer - MRI Dataset/Augmented Data/Augmented/512Pituitary",
             "PMRAM Bangladeshi Brain Cancer - MRI Dataset/PMRAM Bangladeshi Brain Cancer - MRI Dataset/Augmented Data/Augmented/512Normal"]

validImageTypes = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')

tumorFiles, normFiles = [], []

for idx, path in enumerate(dataPaths):
    for filename in os.listdir(path):
        if filename.endswith(validImageTypes):
            image_path = os.path.join(path, filename)
            if idx < 3:
                tumorFiles.append(image_path)
            else:
                normFiles.append(image_path)

allFiles = np.array(tumorFiles + normFiles)
labels = np.array([1] * len(tumorFiles) + [0] * len(normFiles))

In [143]:
X_train, X_test, Y_train, Y_test = train_test_split(allFiles, labels, test_size=0.2, random_state=42)

train_dataset = BrainTumorDataset(X_train, Y_train, transform)
test_dataset = BrainTumorDataset(X_test, Y_test, transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [144]:
model = CNNmodel().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) #learning rate 0.001

In [145]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            predicted = (outputs.squeeze() > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = (outputs.squeeze() > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

In [146]:
# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=10)

# Evaluate the model
evaluate_model(model, test_loader)

Epoch 1/10, Loss: 0.4333, Accuracy: 82.49%
Epoch 2/10, Loss: 0.2961, Accuracy: 88.34%
Epoch 3/10, Loss: 0.2317, Accuracy: 90.53%
Epoch 4/10, Loss: 0.1760, Accuracy: 92.55%
Epoch 5/10, Loss: 0.1240, Accuracy: 94.98%
Epoch 6/10, Loss: 0.0816, Accuracy: 96.92%
Epoch 7/10, Loss: 0.0393, Accuracy: 98.79%
Epoch 8/10, Loss: 0.0236, Accuracy: 99.15%
Epoch 9/10, Loss: 0.0256, Accuracy: 99.23%
Epoch 10/10, Loss: 0.0176, Accuracy: 99.44%


In [147]:
torch.save(model.state_dict(), "brain_tumor_model.pth")

In [148]:
# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load model
model = CNNmodel().to(device)
model.load_state_dict(torch.load('brain_tumor_model.pth', map_location=device))
model.eval()

CNNmodel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (lin1): Linear(in_features=131072, out_features=256, bias=True)
  (lin2): Linear(in_features=256, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)), 
    transforms.ToTensor()
])

# Load image
image_path = 'test_image.jpeg'
image = Image.open(image_path).convert('RGB')
input_tensor = transform(image).unsqueeze(0).to(device)  # add batch dimension

In [160]:
with torch.no_grad():
    output = model(input_tensor)
    prediction = (output.item() > 0.5)

if prediction:
    print("Prediction: Tumor detected")
else:
    print("Prediction: No tumor detected (Normal)")

Prediction: No tumor detected (Normal)
